# **CHURN MODELLING FOR A BANK USING ARTIFICIAL NEURAL NETWORK**

IMPORTING THE LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.3.0'

### **PART 1 - DATA PREPROCESSING**

IMPORTING THE DATASET

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 2:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'Delhi' 'Female' ... 1 1 101348.88]
 [608 'Mumbai' 'Female' ... 0 1 112542.58]
 [502 'Delhi' 'Female' ... 1 0 113931.57]
 ...
 [709 'Delhi' 'Female' ... 0 1 42085.58]
 [772 'Chennai' 'Male' ... 1 0 92888.52]
 [792 'Delhi' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


ENCODING CATEGORICAL DATA

LABEL ENCODING THE "GENDER" COLUMN

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'Delhi' 0 ... 1 1 101348.88]
 [608 'Mumbai' 0 ... 0 1 112542.58]
 [502 'Delhi' 0 ... 1 0 113931.57]
 ...
 [709 'Delhi' 0 ... 0 1 42085.58]
 [772 'Chennai' 1 ... 1 0 92888.52]
 [792 'Delhi' 0 ... 1 0 38190.78]]


ONE HOT ENCODING THE "GEOGRAPHY" COLUMN

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[0.0 1.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [0.0 1.0 0.0 ... 1 0 113931.57]
 ...
 [0.0 1.0 0.0 ... 0 1 42085.58]
 [1.0 0.0 0.0 ... 1 0 92888.52]
 [0.0 1.0 0.0 ... 1 0 38190.78]]


SPLITTING THE DATASET INTO THE TRAINING SET AND TEST SET

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

FEATURE SCALING

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## **PART 2 - BUILDING THE ANN**

INITIALIZING THE ANN

In [12]:
ann = tf.keras.models.Sequential()

ADDING THE INPUT LAYER AND THE FIRST HIDDEN LAYER

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

ADDING THE SECOND HIDDEN LAYER

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

ADDING THE OUTPUT LAYER

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## **PART 3 - TRAINING THE ANN**

COMPILING THE ANN

In [16]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

TRAINING THE ANN ON THE TRAINING SET

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5755 - accuracy: 0.7689
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4791 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4531 - accuracy: 0.7966
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4398 - accuracy: 0.8016
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4323 - accuracy: 0.8065
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4274 - accuracy: 0.8102
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4237 - accuracy: 0.8133
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4207 - accuracy: 0.8142
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4180 - accuracy: 0.8161
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4156 - accura

## **PART 4 - MAKING THE PREDICTIONS AND EVALUATING THE MODEL**

PREDICTING THE RESULT OF A SINGLE OBSERVATION

Use the ANN model to predict if the customer with the following informations will leave the bank:

Geography: Delhi

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer?

SOLUTION:

In [18]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


Therefore, the ANN model predicts that this customer stays in the bank!

PREDICTING THE TEST SET RESULTS

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


MAKING THE CONFUSION MATRIX

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1543   52]
 [ 221  184]]


0.8635